<a href="https://colab.research.google.com/github/ruthetum/Deeplearning-NLP/blob/master/applyRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN을 이용한 텍스트 생성

"a b c d" 라는 문장이 있을 때

데이터 레이블 구성은

a -> b

a, b -> c

a, b, c -> d

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

# 1) 데이터 전처리

경마장 : 있는

경마장에 있는 : 말이

경마장에 있는 말이 : 뛰고

경마장에 있는 말이 뛰고 : 있다

-

그의 : 말이

그의 말이 : 법이다

-

가는 : 말이

가는 말이 : 고와야

가는 말이 고와야 : 오는

가는 말이 고와야 오는 : 말이

가는 말이 고와야 오는 말이 : 곱다

In [0]:
text= "경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다\n"

In [7]:
t = Tokenizer()
t.fit_on_texts([text])
# 정수 인코딩 1부터 시작하므로 size+1
vocab_size = len(t.word_index) + 1 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [8]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [14]:
sequences = list()
index = 0
for line in text.split('\n'):
    index += 1
    print("{} 번째 문장".format(index))
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

    for i in range(1, len(encoded)):
        print("sequence 시작")
        sequence = encoded[:i+1]
        sequences.append(sequence)
        print(sequence)
    print()

print('학습에 사용할 샘플의 개수: %d' % len(sequences))
print(sequences)

1 번째 문장
[2, 3, 1, 4, 5]
sequence 시작
[2, 3]
sequence 시작
[2, 3, 1]
sequence 시작
[2, 3, 1, 4]
sequence 시작
[2, 3, 1, 4, 5]

2 번째 문장
[6, 1, 7]
sequence 시작
[6, 1]
sequence 시작
[6, 1, 7]

3 번째 문장
[8, 1, 9, 10, 1, 11]
sequence 시작
[8, 1]
sequence 시작
[8, 1, 9]
sequence 시작
[8, 1, 9, 10]
sequence 시작
[8, 1, 9, 10, 1]
sequence 시작
[8, 1, 9, 10, 1, 11]

4 번째 문장
[]

학습에 사용할 샘플의 개수: 11
[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [17]:
# 가장 긴 문장 찾아서 padding
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

샘플의 최대 길이 : 6
[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [19]:
# 각 단어의 마지막 단어는 레이블로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [20]:
# 레이블 원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# 2) 모델 설계

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 0s - loss: 2.5099 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 2.4966 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.4835 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4705 - accuracy: 0.1818
Epoch 5/200
1/1 - 0s - loss: 2.4575 - accuracy: 0.1818
Epoch 6/200
1/1 - 0s - loss: 2.4444 - accuracy: 0.1818
Epoch 7/200
1/1 - 0s - loss: 2.4311 - accuracy: 0.2727
Epoch 8/200
1/1 - 0s - loss: 2.4175 - accuracy: 0.2727
Epoch 9/200
1/1 - 0s - loss: 2.4036 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3894 - accuracy: 0.4545
Epoch 11/200
1/1 - 0s - loss: 2.3747 - accuracy: 0.4545
Epoch 12/200
1/1 - 0s - loss: 2.3594 - accuracy: 0.4545
Epoch 13/200
1/1 - 0s - loss: 2.3436 - accuracy: 0.4545
Epoch 14/200
1/1 - 0s - loss: 2.3271 - accuracy: 0.4545
Epoch 15/200
1/1 - 0s - loss: 2.3099 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2919 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2731 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2536 - accuracy: 0.36

In [0]:
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, t, current_word, n):
    # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    init_word = current_word
    sentence = ''

    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0]
        # 데이터 패딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
         # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장
        result = model.predict_classes(encoded, verbose=0)

        # 예측 단어와 동일한 인덱스 단어 찾기
        for word, index in t.word_index.items(): 
            if index == result:
                break
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [28]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측
print(sentence_generation(model, t, '경마장에', 4))
print(sentence_generation(model, t, '그의', 2))
print(sentence_generation(model, t, '가는', 5))

경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
